<a href="https://colab.research.google.com/github/shba007/biosense-ai/blob/feature/indoorplantdiseases_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#credit to https://www.kaggle.com/code/sureshmecad/fruit-images-for-object-detection/notebook

In [ ]:
#editHERE
!unzip /content/indoorplantdiseases.zip -d indoorplantdiseases

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras import applications

In [ ]:
all_img=os.mkdir("/content/all_images")
train=os.mkdir("/content/train")
test=os.mkdir("/content/test")
val=os.mkdir("/content/val")

In [ ]:
# import shutil

# path = '/content/catdogchick'
# i = 0

# for foldername in os.listdir(path):
#   for filename in os.listdir(os.path.join(path,foldername)):
#       print(os.path.join(path,foldername,filename))
#       os.rename(os.path.join(path,foldername,filename), os.path.join(foldername+"_"+str(i)+'.jpg'))
#       shutil.move(os.path.join(foldername+"_"+str(i)+'.jpg'), "/content/all_images")
#       i = i +1

In [ ]:
import os
import random
import shutil

names=[]

class Target:
    def __init__(self, target_path: str, ratio: int, number_of_source_files: int):
        self._target_path = target_path
        self._max = int(number_of_source_files / 100 * ratio)
        self._counter = 0

    @property
    def target_path(self):
        self._counter += 1
        return self._target_path

    def is_maxed_out(self):
        return True if self._counter >= self._max else False


def get_random_target(target_list):
    if len(target_list) > 0:
        return random.choice(target_list)
    else:
        print(target_list)
        # return random.choice([validation, testing, training])

ratio_train=0.8
ratio_val=0
ratio_test=0.2

def placeimg(folder_path,name):
  path_source = folder_path
  os.mkdir(os.path.join('/content/val',name))
  os.mkdir(os.path.join('/content/test',name))
  os.mkdir(os.path.join('/content/train',name))
  source_file_list = os.listdir(path_source)
  testing = Target(repr(os.path.join('/content/test',name))[1:-1], 100*ratio_test, len(source_file_list))
  validation = Target(repr(os.path.join('/content/val',name))[1:-1], 100*ratio_val, len(source_file_list))
  training = Target(repr(os.path.join('/content/train',name))[1:-1], 100*ratio_train, len(source_file_list))

  targets = [validation, testing, training]
  for source_file in source_file_list:
      target = get_random_target(targets)
      if not target== None:
          shutil.move(os.path.join(path_source, source_file), target.target_path)
          for target in targets:
              if target.is_maxed_out():
                  targets.remove(target)

#editHERE
names=["Cercospora","FungalLeafSpots","GrayMold","Powdery Mildew","Rust","Sooty Mold","White Mold","anthracnose","bacterialLeafSpot","healthy"]

#editHERE
placeimg(r'/content/indoorplantdiseases/Cercospora',"Cercospora")
placeimg(r'/content/indoorplantdiseases/FungalLeafSpots',"FungalLeafSpots")
placeimg(r'/content/indoorplantdiseases/GrayMold',"GrayMold")
placeimg(r'/content/indoorplantdiseases/Powdery Mildew',"Powdery Mildew")
placeimg(r'/content/indoorplantdiseases/Rust',"Rust")
placeimg(r'/content/indoorplantdiseases/Sooty Mold',"Sooty Mold")
placeimg(r'/content/indoorplantdiseases/White Mold',"White Mold")
placeimg(r'/content/indoorplantdiseases/anthracnose',"anthracnose")
placeimg(r'/content/indoorplantdiseases/bacterialLeafSpot',"bacterialLeafSpot")
placeimg(r'/content/indoorplantdiseases/healthy',"healthy")

# def placeimg(folder_path,name):
#   path_source = folder_path
#   source_file_list = os.listdir(path_source)
#   validation = Target(r'/content/val', 100*ratio_val, len(source_file_list))
#   testing = Target(r'/content/test', 100*ratio_test, len(source_file_list))
#   training = Target(r'/content/train', 100*ratio_train, len(source_file_list))

#   targets = [validation, testing, training]
#   for source_file in source_file_list:
#       target = get_random_target(targets)
#       shutil.move(os.path.join(path_source, source_file), target.target_path)
#       for target in targets:
#           if target.is_maxed_out():
#               targets.remove(target)
# placeimg(r"/content/all_images","cat")

In [ ]:
trdata = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
traindata = trdata.flow_from_directory(directory="/content/train",target_size=(224,224))
tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory="/content/test", target_size=(224,224))

In [ ]:
# VGG=keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet")
# VGG.trainable=False
from keras.applications import VGG16
from keras.layers import Dropout
from keras.models import Model

VGG = VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet")
VGG.trainable=False

# Store the fully connected layers
fc1 = VGG.layers[-3]
fc2 = VGG.layers[-2]
predictions = VGG.layers[-1]

# Create the dropout layers
dropout1 = Dropout(0.5)
dropout2 = Dropout(0.5)

# Reconnect the layers
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)

# Create a new model
VGG = Model(inputs=VGG.input, outputs=predictors)

In [ ]:
model = Sequential([
    VGG,
    Flatten(),
    # Dense(units=256,activation="relu"),
    # Dense(units=256,activation="relu"),
    #editHERE
    Dense(units=10,activation="softmax")
])
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#editHERE
steps_per_epoch= (752+321)//64

# hist=model.fit_generator(steps_per_epoch=steps_per_epoch,generator=traindata,validation_data=testdata,validation_steps=10,epochs=5)
hist=model.fit_generator(generator=traindata,validation_data=testdata,epochs=100)
#editHERE
model.save("indoorplantdiseases.h5")

In [ ]:
import matplotlib.pyplot as plt
import keras.utils as image
import matplotlib
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from keras.models import load_model
import pandas as pd
#editHERE
saved_model = load_model("indoorplantdiseases.h5")

In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
img = image.load_img("/content/download (1).jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
output = saved_model.predict(img)
print(output)
print(output[0][np.where(output == (np.argmax(output)))[0]])
# if output[0][0] > output[0][1]:
#     print("cat")
# else:
#     print('dog')
print(names[np.argmax(output[0])])


In [ ]:
probabilities = saved_model.predict_generator(generator=testdata)
probabilities[1]

In [ ]:
test_labels=testdata[0][1]
len(test_labels[1])

In [ ]:
rounded_labels=np.argmax(test_labels, axis=1)
rounded_labels[1]
rounded_labels
# test_labels=pd.get_dummies(test_labels, columns = names)

In [ ]:
# rounded_predictions = saved_model.predict_classes(test_images, batch_size=128, verbose=0)
predictions = (saved_model.predict(testdata) > 0.5).astype("int32")
predictions=np.argmax(predictions, axis=1)
predictions

In [ ]:
cm = confusion_matrix(rounded_labels, predictions[:len(rounded_labels)])
cm